<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

<a name="02_prepare"></a>
# 2. Prepare the Data with *pandas* and *NumPy*
<hr>

The initial data, in its raw form, includes variabilities that will become problematic when we process it through a model to find patterns.  The variabilities in the data that we want to mitigate are:

* Measurement values have widely varying ranges that require normalization
* Not every measurement is taken during every observation, resulting in data gaps
* The number of observations per encounter varies widely

In this notebook, we will introduce techniques to address these concerns and prepare the data for training. 

2.1 [**Normalize the Data**](#02_normalize)<br>
2.2 [**Fill Data Gaps**](#02_gaps)<br>
2.3 [**Pad Variable Length Sequences**](#02_pad)<br>
&nbsp; &nbsp; &nbsp;2.3.1 [Exercise: Padded Variable over all Patient Encounters](#02_ex_pad)<br>
2.4 [**Save a *NumPy* Array**](#02_save)<br>
[**Solutions**](#02_solutions)<br>

First, import the libraries for the new notebook, this time including _Keras_, our deep learning framework.  We also need to reload the data saved at the end of the previous notebook as `.pkl` (pickle) files.

In [ ]:
import os
import numpy as np          
import pandas as pd              
import matplotlib.pyplot as plt  
import random
import tensorflow.keras as keras

# configure notebook to display plots
%matplotlib inline

# set up user paths
data_dir = '/dli/task/data/hx_series'
csv_dir = '/dli/task/csv'

In [ ]:
# Fetch the DataFrame's loaded in the problem setup
X_train=pd.read_pickle('X_train.pkl')
y_train=pd.read_pickle('y_train.pkl')
X_valid=pd.read_pickle('X_valid.pkl')
y_valid=pd.read_pickle('y_valid.pkl')

<a name="02_normalize"></a>
## 2.1 Normalize the Data
We normalize each observed feature / variable by subtracting its mean and dividing the result by the standard deviation.  

Why?

We want small variations in one variable to be treated with the same emphasis as HUGE variations of another. Keep in mind that the network just sees a bunch of numbers - it doesn't actually "know" anything about predictors, factors, variables, obervations and so on and so forth. Emperically, normalization seems to facilitate training but this kind of normalization is probably not appropriate for multimodal data (or non-Gaussian data in general).

Lets take a look at both before and after normalization of a single encounter example:

In [ ]:
# Before normalization
X_train.loc[8,['Age','Heart rate (bpm)','PulseOximetry','Weight',
    'SystolicBP','DiastolicBP','Respiratory rate (bpm)',
    'MotorResponse','Capillary refill rate (sec)']]

In [ ]:
# create file path for csv file with metadata about variables
metadata = os.path.join(csv_dir, 'ehr_features.csv')

# read in variables from csv file (using pandas) since each varable there is tagged with a category
variables = pd.read_csv(metadata, index_col=0)

# next, select only variables of a particular category for normalization
normvars = variables[variables['type'].isin(['Interventions', 'Labs', 'Vitals'])]

# finally, iterate over each variable in both training and validation data
for vId, dat in normvars.iterrows():
    
    X_train[vId] = X_train[vId] - dat['mean']
    X_valid[vId] = X_valid[vId] - dat['mean']
    X_train[vId] = X_train[vId] / (dat['std'] + 1e-12)
    X_valid[vId] = X_valid[vId] / (dat['std'] + 1e-12)
    

Take a quick look at some the updated data for a single encounter.

In [ ]:
# After normalization
X_train.loc[8,['Age','Heart rate (bpm)','PulseOximetry','Weight',
    'SystolicBP','DiastolicBP','Respiratory rate (bpm)',
    'MotorResponse','Capillary refill rate (sec)']]

<a name="02_gaps"></a>
## 2.2 Fill Data Gaps
Having normalized the data, we still need to fill in all the data gaps since not every variable was observed during each observation in the patient encounter.  For example, lets take a look at the gaps in a "Heart rate" encounter by plotting it.

In [ ]:
# Before filling gaps
X_train.loc[8, "Heart rate (bpm)"].plot()
plt.title("Normalized Not Filled")
plt.ylabel("Heart rate (bpm)")
plt.xlabel("Hours since first encounter")
plt.show()

Filling in the gaps of missing data is a very active area of research and there is currently no standard practice for time series analysis using deep learning.  For this tutorial, we will simply forward fill existing measurements for each patient, and fill any variable entries with no previous measurement to 0 as illustrated below.

<img src="images/imputation_diagram.svg" width="800" height="200">

In [ ]:
# first select variables which will be filled in
fillvars = variables[variables['type'].isin(['Vitals', 'Labs'])].index

# next forward fill any missing values with more recently observed value
X_train[fillvars] = X_train.groupby(level=0)[fillvars].ffill()
X_valid[fillvars] = X_valid.groupby(level=0)[fillvars].ffill()

# finally, fill in any still missing values with 0 (i.e. values that could not be filled forward)
X_train.fillna(value=0, inplace=True)
X_valid.fillna(value=0, inplace=True)

Now lets have a look at the "Heart rate" variable after data normalization and missing values have been filled in.|

In [ ]:
# After filling gaps
X_train.loc[8, "Heart rate (bpm)"].plot()
plt.title("Normalized and Filled")
plt.ylabel("Heart rate (bpm)")
plt.xlabel("Hours since first encounter")
plt.show()

Also, try dumping the X_train vector to the screen again and you will see that all those NaN values have been filled in with zeros.

In [ ]:
X_train

One more thing to note, is that so far we've kept the data set as a _pandas_ DataFrame type.  In the next part, we will change it to a _NumPy_ array.  You can check this as follows:

In [ ]:
type(X_train)

<a name="02_pad"></a>
## 2.3 Pad Variable Length Sequences
<img style="float: right;" src="images/nobs_hist.png" width="400">
<br>
The final data preparation task is to pad every patient encounter so that all encounters have the same number of observations. 
Note from the histogram, that there are many encounters with less than 100 observation vectors. Therefore, we are going to zero pad each encounter (i.e. insert rows of zeros).

In [ ]:
from tensorflow.keras.preprocessing import sequence

# max number of sequence length
maxlen = 500

# get a list of unique patient encounter IDs
teId = X_train.index.levels[0]
veId = X_valid.index.levels[0]

# pad every patient sequence with 0s to be the same length, 
# then transforms the list of sequences to one numpy array
# this is for efficient minibatching and GPU computations 
X_train = [X_train.loc[patient].values for patient in teId]
y_train = [y_train.loc[patient].values for patient in teId]

X_train = sequence.pad_sequences(X_train, dtype='float32', maxlen=maxlen, padding='post', truncating='post')
y_train = sequence.pad_sequences(y_train, dtype='float32', maxlen=maxlen, padding='post', truncating='post')

# repeat for the validation data

X_valid = [X_valid.loc[patient].values for patient in veId]
y_valid = [y_valid.loc[patient].values for patient in veId]

X_valid = sequence.pad_sequences(X_valid, dtype='float32', maxlen=maxlen, padding='post', truncating='post')
y_valid = sequence.pad_sequences(y_valid, dtype='float32', maxlen=maxlen, padding='post', truncating='post')

Okay, a lot just happened here:  

1. We converted the _pandas_ DataFrame into a Python list, which contained lists of values (a list of list of values).  
2. Using  `keras.preprocessing.sequence.pad_sequences`, we converted the value lists into a `numpy.array` of type `float32`, having a maximum length of 500.  
3. If the patient encounter didn't have 500 encounters (most don't, see previous histogram) then we apply `padding='post'`, which says to pad with zeros.  That is, add extra rows (observation vectors) of all zeros.
4. The option `truncating='post'` just says: if there are more than 500 observations, then take the first 500 and drop everything after.  

Together, this says: **force patient encounter records of dimension 500x265 and use zero padding to inflate the size if needed**.  We could do something similar in _pandas_ but not as succinctly. This only requires a single command using _NumPy_.

In [ ]:
# print the shape of the array which will be used by the network
# the shape is of the form (# of encounters, length of sequence, # of features)
print("X_train shape: %s | y_train shape: %s" % (str(X_train.shape), str(y_train.shape)))
print("X_valid shape: %s | y_valid shape: %s" % (str(X_valid.shape), str(y_valid.shape)))

Note that the type of X_train has changed to ```numpy.ndarray```:

In [ ]:
type(X_train)

Now we can plot the full patient encounter as a matrix plot.  Try a few times to get a feel for what the charts look like.

In [ ]:
# figure out how many encounters we have
numencnt = X_train.shape[0]

# choose a random patient encounter to plot
ix = random.randint(0,5000) #Try a few different index values between 0 and 4999
print('ix = {}'.format(ix))

# plot a matrix of observation values
plt.title("Patient Encounter Matrix")
plt.pcolor(np.transpose(X_train[ix,:,:]))
plt.ylabel("variable")
plt.xlabel("time/observation")
plt.ylim(0,265)
plt.colorbar()
plt.show()

Keep in mind here that these data are zero padded by ```sequence.pad_sequences```.  Try a few different index values between 0 and 4999 to get a feel for what the matrix plots look like. These matricies are what will be fed as input into the LSTM model for training. Notice that we can plot a variable in a similar fashion by selecting along the third axis instead of the first axis. This provides a view of a particular variable over all patient encounters. Give it a try!

<a name='02_ex_pad'></a>
### 2.3.1 Exercise: Padded Variable over all Patient Encounters

Visualize a particular variable across all encounters with the following steps.  You can also check the [solution](#02_sol_pad) if you wish.
1. Choose a variable to examine from [ehr_features.csv](csv/ehr_features.csv)
2. Create a matrix of encounters vs time/observation
3. Plot the matrix

In [ ]:
# TODO Step 1 set the variable value (feature number)
varnum = None  #FIXME

# TODO Step 2 Create a matrix of encounters vs time/observation
# Hint: Select along the 3rd axis
varmatrix = []  #FIXME

# Step 3 Plot the matrix
try:
    plt.title("Variable Matrix")
    plt.pcolor(varmatrix) 
    plt.ylabel("time/observation")
    plt.xlabel("encounter")
    plt.ylim(0,600)
    plt.colorbar()
    plt.show()
except Exception as e:
    print('ERROR found: {}'.format(e))

<a name="02_save"></a>
## 2.4 Save a *NumPy* Arrays

Before moving to the next notebook, save the *NumPy* arrays to reload into the next notebook.

In [ ]:
# Save the prepared numpy arrays for use in other notebooks
np.save('X_train_prepared.npy',X_train,allow_pickle=False)
np.save('y_train_prepared.npy',y_train,allow_pickle=False)
np.save('X_valid_prepared.npy',X_valid,allow_pickle=False)
np.save('y_valid_prepared.npy',y_valid,allow_pickle=False)

<font color='green' size=10>Excellent work!</font><br>
Now that we've prepared the data, we'll build and train our model.  Move on to the [next notebook](03_BuildModel.ipynb).

<img src="images/DLI Header.png" alt="Header" style="width: 400px;"/>

<hr>

<a name='02_solutions'></a>
# Solutions
<a name='02_sol_pad'></a>
### 2.3.1 Exercise: Padded Variable over all Patient Encounters 
[Jump back to Exercise 2.3.1](#02_ex_pad)

In [ ]:
# TODO Step 1 set the variable value (feature number)
varnum = 115 #FIXME  (any number between 0 and 264, inclusive)

# TODO Step 2 Create a matrix of encounters vs time/observation
# Hint: Select along the 3rd axis
varmatrix = np.transpose(X_train[:,:,varnum])  #FIXME

# Step 3 Plot the matrix
try:
    plt.title("Variable Matrix")
    plt.pcolor(varmatrix) 
    plt.ylabel("time/observation")
    plt.xlabel("encounter")
    plt.ylim(0,600)
    plt.colorbar()
    plt.show()
except Exception as e:
    print('ERROR found: {}'.format(e))